In [3]:
import torch
from torch import nn
from torch.nn import functional as F
#functional里面封装了一些没有包含参数的函数
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))
#nn.Sequential定义了一个特殊的module

X = torch.rand(2, 20)
net(X)

tensor([[-0.0362, -0.0573,  0.1992,  0.0994, -0.0858,  0.1891, -0.0139, -0.0217,
          0.1455,  0.1645],
        [-0.2436, -0.1586,  0.1401,  0.0990,  0.0318,  0.1750,  0.0207, -0.1341,
          0.1067, -0.0930]], grad_fn=<AddmmBackward0>)

In [4]:
#实现多层感知机
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X)

tensor([[ 0.2915, -0.0053, -0.4137,  0.2131, -0.0870, -0.2433, -0.3031, -0.2113,
         -0.0466, -0.1544],
        [ 0.1851,  0.1030, -0.2574, -0.1906, -0.0111, -0.1319,  0.0333, -0.1411,
         -0.0709, -0.1760]], grad_fn=<AddmmBackward0>)

## 通过继承nn.Module可以定义更灵活的Sequential的类
### 如：

In [ ]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
        
    def forward(self, X):
        for block in self.modules.values():
            X = block[X]
        return X
net = MySequential(nn.Linear(20,256), nn.ReLU(),nn.Linear(256,10))
net(X)

In [10]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20,20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1 :
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(-0.1196, grad_fn=<SumBackward0>)

## 混合搭配各种组合块的方法

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64), nn.ReLU(),
                                 nn.Linear(64,32), nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16,20), FixedHiddenMLP())
chimera(X)

tensor(-0.0248, grad_fn=<SumBackward0>)